In [1]:
import torch
import torchvision
import datetime
from torchvision import transforms
from torchvision.models import resnet50
from PIL import Image
import time
import pytz
# from jtop import jtop
# jetson = jtop()
# jetson.start()
# print("Before loading model: ",jetson.stats)
def read_classes():
    """
    Load the ImageNet class names.
    """
    with open("imagenet-classes.txt", "r") as f:
        categories = [s.strip() for s in f.readlines()]
    return categories

print("Before loading model: ",datetime.datetime.now(pytz.timezone('Europe/London')))
# Load the pre-trained ResNet model
model = resnet50(pretrained=True)
# print(model)
categories = read_classes()
model.eval()  # Set the model to evaluation mode



Before loading model:  2024-10-18 18:13:40.096077+01:00


Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100.0%


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [19]:
# Load and preprocess an example image
print("Loading image: ",datetime.datetime.now(pytz.timezone('Europe/London')))
image_path = 'dog_1.jpg'
input_image = Image.open(image_path).convert('RGB')
# print("Image transformation: ",jetson.stats)

Loading image:  2024-10-18 18:25:34.174454+01:00


In [20]:
print("Image transformation: ",datetime.datetime.now(pytz.timezone('Europe/London')))
# Define the image transformation pipeline
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Apply transformations to the input image
input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0)  # Add a batch dimension


Image transformation:  2024-10-18 18:25:37.744258+01:00


In [21]:
# Make the prediction
with torch.no_grad():
    start_time = time.time()
    print("During prediction: ", datetime.datetime.now(pytz.timezone('Europe/London')))
    # print("During prediction: ",jtop().stats)
    output = model(input_batch)
    end_time = time.time()

# Get the predicted class index
# get the softmax probabilities
probabilities = torch.nn.functional.softmax(output[0], dim=0)
# check the top 5 categories that are predicted
top5_prob, top5_catid = torch.topk(probabilities, 1)

# Print the predicted class label and inference time
inference_time = end_time - start_time
print(f'Inference time: {inference_time:.4f} seconds')
print(f"Accuracy: {top5_prob[0].item()*100:.3f}%")
print(f"Predicted class:{categories[top5_catid[0]]}")      

During prediction:  2024-10-18 18:25:42.362320+01:00
Inference time: 1.0365 seconds
Accuracy: 88.961%
Predicted class:Newfoundland, Newfoundland dog


In [5]:
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Total trainable parameters: {total_params}')

Total trainable parameters: 25557032


In [6]:
import torch

# Load the pre-trained Mobilenetv3 model
model = resnet50(pretrained=True)

# Define input size (224x224x3) for Mobilenetv3
input_size = (224, 224, 3)

# Function to calculate FLOPs for the model
def count_flops(model, input_size):
    # Define a tensor of appropriate size
    input_tensor = torch.randn(1, *input_size)
    
    # Switch to evaluation mode
    model.eval()
    
    # Move model to appropriate device
    device = next(model.parameters()).device
    input_tensor = input_tensor.to(device)
    model.to(device)
    
    # Iterate through model's layers
    flops = 0
    for module in model.modules():
        if isinstance(module, torch.nn.Conv2d):
            # For convolutional layers
            output_size = (input_size[0] - module.kernel_size[0] + 2 * module.padding[0]) // module.stride[0] + 1
            output_size = (output_size - module.kernel_size[1] + 2 * module.padding[1]) // module.stride[1] + 1
            flops += module.in_channels * module.out_channels * module.kernel_size[0] * module.kernel_size[1] * output_size * output_size
            input_size = (output_size, output_size, module.out_channels)
        elif isinstance(module, torch.nn.MaxPool2d):
            # For max pooling layers
            output_size = (input_size[0] - module.kernel_size) // module.stride + 1
            flops += input_size[2] * output_size * output_size
            input_size = (output_size, output_size, input_size[2])
        elif isinstance(module, torch.nn.Linear):
            # For fully connected layers
            flops += module.in_features * module.out_features
            input_size = (module.out_features,)
    
    return flops

# Calculate FLOPs
total_flops = count_flops(model, input_size)
print("Total FLOPs:", total_flops)

Total FLOPs: 247473728


Total FLOPs: 247473728
Before loading model:  2024-10-18 18:13:40.096077+01:00

0: 1.0

Loading image:  2024-10-18 18:14:52.584886+01:00
Image transformation:  2024-10-18 18:14:55.387720+01:00
During prediction:  2024-10-18 18:14:57.926422+01:00
tensor([232])
Predicted class: Dogs
Inference time: 0.9052 seconds
Accuracy: 19.451%
class:Border collie

datetime.datetime(2024, 10, 18, 18, 13, 40, 307499)
datetime.datetime(2024, 10, 18, 18, 13, 41, 309383)
datetime.datetime(2024, 10, 18, 18, 13, 42, 316040)
datetime.datetime(2024, 10, 18, 18, 13, 43, 307670)
datetime.datetime(2024, 10, 18, 18, 13, 44, 316252)
datetime.datetime(2024, 10, 18, 18, 13, 45, 319497)
datetime.datetime(2024, 10, 18, 18, 13, 46, 320285)
datetime.datetime(2024, 10, 18, 18, 13, 47, 317936)
datetime.datetime(2024, 10, 18, 18, 13, 48, 318607)
datetime.datetime(2024, 10, 18, 18, 13, 49, 320045)
datetime.datetime(2024, 10, 18, 18, 13, 50, 319053)
datetime.datetime(2024, 10, 18, 18, 13, 51, 319938)
datetime.datetime(2024, 10, 18, 18, 13, 52, 321799)
datetime.datetime(2024, 10, 18, 18, 13, 53, 326427)
datetime.datetime(2024, 10, 18, 18, 13, 54, 324620)
datetime.datetime(2024, 10, 18, 18, 13, 55, 336766)
datetime.datetime(2024, 10, 18, 18, 13, 56, 325393)
datetime.datetime(2024, 10, 18, 18, 13, 57, 327153)
datetime.datetime(2024, 10, 18, 18, 13, 58, 332215)
datetime.datetime(2024, 10, 18, 18, 13, 59, 336764)
datetime.datetime(2024, 10, 18, 18, 14, 00, 330026)
datetime.datetime(2024, 10, 18, 18, 14, 01, 340303)
datetime.datetime(2024, 10, 18, 18, 14, 02, 337380)
datetime.datetime(2024, 10, 18, 18, 14, 03, 339589)
datetime.datetime(2024, 10, 18, 18, 14, 04, 346921)
datetime.datetime(2024, 10, 18, 18, 14, 05, 341100)
datetime.datetime(2024, 10, 18, 18, 14, 06, 338369)
datetime.datetime(2024, 10, 18, 18, 14, 07, 343667)
datetime.datetime(2024, 10, 18, 18, 14, 08, 338954)
datetime.datetime(2024, 10, 18, 18, 14, 09, 346450)
datetime.datetime(2024, 10, 18, 18, 14, 10, 350250)
datetime.datetime(2024, 10, 18, 18, 14, 11, 349287)
datetime.datetime(2024, 10, 18, 18, 14, 12, 341730)
datetime.datetime(2024, 10, 18, 18, 14, 13, 340068)
datetime.datetime(2024, 10, 18, 18, 14, 14, 341322)
datetime.datetime(2024, 10, 18, 18, 14, 15, 341286)
datetime.datetime(2024, 10, 18, 18, 14, 16, 340735)
datetime.datetime(2024, 10, 18, 18, 14, 17, 348287)
datetime.datetime(2024, 10, 18, 18, 14, 18, 346365)
datetime.datetime(2024, 10, 18, 18, 14, 19, 342858)
datetime.datetime(2024, 10, 18, 18, 14, 20, 343660)
datetime.datetime(2024, 10, 18, 18, 14, 21, 345901)
datetime.datetime(2024, 10, 18, 18, 14, 22, 346058)
datetime.datetime(2024, 10, 18, 18, 14, 23, 346948)
datetime.datetime(2024, 10, 18, 18, 14, 24, 349297)
datetime.datetime(2024, 10, 18, 18, 14, 25, 349565)
datetime.datetime(2024, 10, 18, 18, 14, 26, 350795)
datetime.datetime(2024, 10, 18, 18, 14, 27, 351315)
datetime.datetime(2024, 10, 18, 18, 14, 28, 351894)
datetime.datetime(2024, 10, 18, 18, 14, 29, 352372)
datetime.datetime(2024, 10, 18, 18, 14, 30, 353386)
datetime.datetime(2024, 10, 18, 18, 14, 31, 355101)
datetime.datetime(2024, 10, 18, 18, 14, 32, 356510)
datetime.datetime(2024, 10, 18, 18, 14, 33, 357109)
datetime.datetime(2024, 10, 18, 18, 14, 34, 357740)
datetime.datetime(2024, 10, 18, 18, 14, 35, 359095)
datetime.datetime(2024, 10, 18, 18, 14, 36, 361487)
datetime.datetime(2024, 10, 18, 18, 14, 37, 361398)
datetime.datetime(2024, 10, 18, 18, 14, 38, 371114)
datetime.datetime(2024, 10, 18, 18, 14, 39, 376105)
datetime.datetime(2024, 10, 18, 18, 14, 40, 377228)
datetime.datetime(2024, 10, 18, 18, 14, 41, 379336)
datetime.datetime(2024, 10, 18, 18, 14, 42, 379187)
datetime.datetime(2024, 10, 18, 18, 14, 43, 379159)
datetime.datetime(2024, 10, 18, 18, 14, 44, 369457)
datetime.datetime(2024, 10, 18, 18, 14, 45, 378911)
datetime.datetime(2024, 10, 18, 18, 14, 46, 387683)
datetime.datetime(2024, 10, 18, 18, 14, 47, 384992)
datetime.datetime(2024, 10, 18, 18, 14, 48, 389141)
datetime.datetime(2024, 10, 18, 18, 14, 49, 390301)
datetime.datetime(2024, 10, 18, 18, 14, 50, 387849)
datetime.datetime(2024, 10, 18, 18, 14, 51, 384437)
datetime.datetime(2024, 10, 18, 18, 14, 52, 390137)
datetime.datetime(2024, 10, 18, 18, 14, 53, 395730)
datetime.datetime(2024, 10, 18, 18, 14, 54, 389025)
datetime.datetime(2024, 10, 18, 18, 14, 55, 401534)
datetime.datetime(2024, 10, 18, 18, 14, 56, 395010)
datetime.datetime(2024, 10, 18, 18, 14, 57, 397681)
datetime.datetime(2024, 10, 18, 18, 14, 58, 433099)

Average CPU Temperature: 33.11°C
Average GPU Temperature: 33.80°C
Average Power CPU: 1228.01mW
Average Power GPU: 1228.01mW
Average Power TOT: 2830.39mW
Average RAM: 0.4962

{'Avg_CPU_temp': 33.11392405063291, 'Avg_GPU_temp': 33.79746835443038, 'Avg_Power_CPU': 1228.012658227848, 'Avg_Power_GPU': 1228.012658227848, 'Avg_CPU_RAM': 0.49615734017180224}

1: cat_0

Loading image:  2024-10-18 18:20:10.711479+01:00
Image transformation:  2024-10-18 18:20:13.330517+01:00
During prediction:  2024-10-18 18:20:16.040559+01:00
Inference time: 0.6686 seconds
Accuracy: 18.899%
Predicted class:pop bottle, soda bottle

datetime.datetime(2024, 10, 18, 18, 20, 10, 823017)
datetime.datetime(2024, 10, 18, 18, 20, 11, 837530)
datetime.datetime(2024, 10, 18, 18, 20, 12, 838801)
datetime.datetime(2024, 10, 18, 18, 20, 13, 834767)
datetime.datetime(2024, 10, 18, 18, 20, 14, 836573)
datetime.datetime(2024, 10, 18, 18, 20, 15, 846941)
datetime.datetime(2024, 10, 18, 18, 20, 16, 838221)
datetime.datetime(2024, 10, 18, 18, 20, 17, 846762)

Average CPU Temperature: 32.19°C
Average GPU Temperature: 32.94°C
Average Power CPU: 857.88mW
Average Power GPU: 857.88mW
Average Power TOT: 2325.88mW
Average RAM: 0.4969

{'Avg_CPU_temp': 32.1875, 'Avg_GPU_temp': 32.9375, 'Avg_Power_CPU': 857.875, 'Avg_Power_GPU': 857.875, 'Avg_CPU_RAM': 0.4969140775120466}

2: cat_1

Loading image:  2024-10-18 18:21:36.257550+01:00
Image transformation:  2024-10-18 18:21:38.402992+01:00
During prediction:  2024-10-18 18:21:41.966783+01:00
Inference time: 0.6865 seconds
Accuracy: 64.667%
Predicted class:tabby, tabby cat

datetime.datetime(2024, 10, 18, 18, 21, 36, 949844)
datetime.datetime(2024, 10, 18, 18, 21, 37, 955878)
datetime.datetime(2024, 10, 18, 18, 21, 38, 946865)
datetime.datetime(2024, 10, 18, 18, 21, 39, 959221)
datetime.datetime(2024, 10, 18, 18, 21, 40, 961720)
datetime.datetime(2024, 10, 18, 18, 21, 41, 966502)
datetime.datetime(2024, 10, 18, 18, 21, 42, 950842)

Average CPU Temperature: 32.00°C
Average GPU Temperature: 32.71°C
Average Power CPU: 727.00mW
Average Power GPU: 727.00mW
Average Power TOT: 2070.14mW
Average RAM: 0.4954

{'Avg_CPU_temp': 32.0, 'Avg_GPU_temp': 32.714285714285715, 'Avg_Power_CPU': 727.0, 'Avg_Power_GPU': 727.0, 'Avg_CPU_RAM': 0.4954317979298025}

3: dog_0

Loading image:  2024-10-18 18:22:55.936811+01:00
Image transformation:  2024-10-18 18:22:58.133207+01:00
During prediction:  2024-10-18 18:23:00.926361+01:00
Inference time: 0.8191 seconds
Accuracy: 95.679%
Predicted class:malamute, malemute, Alaskan malamute

datetime.datetime(2024, 10, 18, 18, 22, 55, 063147)
datetime.datetime(2024, 10, 18, 18, 22, 56, 048290)
datetime.datetime(2024, 10, 18, 18, 22, 57, 052637)
datetime.datetime(2024, 10, 18, 18, 22, 58, 076728)
datetime.datetime(2024, 10, 18, 18, 22, 59, 071943)
datetime.datetime(2024, 10, 18, 18, 23, 00, 065504)
datetime.datetime(2024, 10, 18, 18, 23, 01, 156040)
datetime.datetime(2024, 10, 18, 18, 23, 02, 053621)

Average CPU Temperature: 32.06°C
Average GPU Temperature: 32.94°C
Average Power CPU: 953.12mW
Average Power GPU: 953.12mW
Average Power TOT: 2409.62mW
Average RAM: 0.4953

{'Avg_CPU_temp': 32.0625, 'Avg_GPU_temp': 32.9375, 'Avg_Power_CPU': 953.125, 'Avg_Power_GPU': 953.125, 'Avg_CPU_RAM': 0.49530589469950037}

4: cat_2

Loading image:  2024-10-18 18:24:20.932421+01:00
Image transformation:  2024-10-18 18:24:23.105828+01:00
During prediction:  2024-10-18 18:24:25.821072+01:00
Inference time: 0.7733 seconds
Accuracy: 99.542%
Predicted class:Persian cat

datetime.datetime(2024, 10, 18, 18, 24, 20, 179869)
datetime.datetime(2024, 10, 18, 18, 24, 21, 174983)
datetime.datetime(2024, 10, 18, 18, 24, 22, 185929)
datetime.datetime(2024, 10, 18, 18, 24, 23, 175934)
datetime.datetime(2024, 10, 18, 18, 24, 24, 183553)
datetime.datetime(2024, 10, 18, 18, 24, 25, 177050)
datetime.datetime(2024, 10, 18, 18, 24, 26, 271601)

Average CPU Temperature: 32.07°C
Average GPU Temperature: 32.86°C
Average Power CPU: 982.86mW
Average Power GPU: 982.86mW
Average Power TOT: 2468.14mW
Average RAM: 0.4954

{'Avg_CPU_temp': 32.07142857142857, 'Avg_GPU_temp': 32.857142857142854, 'Avg_Power_CPU': 982.8571428571429, 'Avg_Power_GPU': 982.8571428571429, 'Avg_CPU_RAM': 0.4954022357457483}

5: dog_1

Loading image:  2024-10-18 18:25:34.174454+01:00
Image transformation:  2024-10-18 18:25:37.744258+01:00
During prediction:  2024-10-18 18:25:42.362320+01:00
Inference time: 1.0365 seconds
Accuracy: 88.961%
Predicted class:Newfoundland, Newfoundland dog

datetime.datetime(2024, 10, 18, 18, 25, 34, 268936)
datetime.datetime(2024, 10, 18, 18, 25, 35, 300201)
datetime.datetime(2024, 10, 18, 18, 25, 36, 287931)
datetime.datetime(2024, 10, 18, 18, 25, 37, 288182)
datetime.datetime(2024, 10, 18, 18, 25, 38, 290131)
datetime.datetime(2024, 10, 18, 18, 25, 39, 295299)
datetime.datetime(2024, 10, 18, 18, 25, 40, 289772)
datetime.datetime(2024, 10, 18, 18, 25, 41, 292486)
datetime.datetime(2024, 10, 18, 18, 25, 42, 295274)
datetime.datetime(2024, 10, 18, 18, 25, 43, 352813)

Average CPU Temperature: 31.90°C
Average GPU Temperature: 32.35°C
Average Power CPU: 775.60mW
Average Power GPU: 775.60mW
Average Power TOT: 2171.40mW
Average RAM: 0.4952

{'Avg_CPU_temp': 31.9, 'Avg_GPU_temp': 32.35, 'Avg_Power_CPU': 775.6, 'Avg_Power_GPU': 775.6, 'Avg_CPU_RAM': 0.49522817079059134}